In [1]:
from google.colab import drive

drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
%cd 'gdrive/My Drive'

/content/gdrive/My Drive


In [3]:
pip install konlpy

     |████████████████████████████████| 19.4MB 140kB/s 
     |████████████████████████████████| 491kB 45.5MB/s 
     |████████████████████████████████| 92kB 10.0MB/s 
  Created wheel for JPype1: filename=JPype1-0.7.1-cp36-cp36m-linux_x86_64.whl size=2454808 sha256=de9778c09fce1be9d83592f81fea5a13f1f2c3d08199dabb590f643f9953d9e2
  Stored in directory: /root/.cache/pip/wheels/b0/49/6a/4393ef8542c00becf80691bd242693db9e263d6e499323a984
Successfully built JPype1
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3
  Found existing installation: tweepy 3.6.0
    Uninstalling tweepy-3.6.0:
      Successfully uninstalled tweepy-3.6.0


In [0]:
# OS 라이브러리 임포트
import os
# 정규표현식 라이브러리 임포트
import re

# scikit-learn 라이브러리 임포트
from sklearn import datasets, model_selection
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

# 형태소분석기 라이브러리
from konlpy.tag import Hannanum
from konlpy.tag import Kkma

# pandas 라이브러리 임포트
import pandas as pd

# numpy 라이브러리 임포트
import numpy as np

In [0]:
dir_prefix = 'news data/'
target_dir = 'HKIB-20000'
cat_dirs = ['health', 'economy', 'science', 'education', 'culture', 'society', 'industry', 'leisure', 'politics']
cat_prefixes = ['건강', '경제', '과학', '교육', '문화', '사회', '산업', '여가', '정치']

In [0]:
# 데이터 정리
files = os.listdir(dir_prefix + target_dir)

# 5분할된 텍스트 파일을 각각 처리
for file in files:
    # txt형식의 데이터가 담긴 파일만 처리
    if not file.endswith('.txt'):
        continue
    
    # 각 텍스트 파일을 처리
    with open(dir_prefix + target_dir + '/' + file) as currfile:
        doc_cnt = 0 # 기사 갯수
        docs = [] # 기사 배열
        curr_doc = None # 기사가 들어갈 임시변수
        
        # 기사 단위로 분할하여 리스트를 생성
        for curr_line in currfile:
            if curr_line.startswith('@DOCUMENT'): # 각 기사는 @DOCUMENT로 구분되어 있음
                if curr_doc is not None: # 기사가 들어갈 임시 변수에 값이 있다면
                    docs.append(curr_doc) # curr_doc에 값 입력
                curr_doc = curr_line
                doc_cnt = doc_cnt + 1
                continue
            curr_doc = curr_doc + curr_line # curr_doc에 기사 단위로 누적
        
        # 각 기사를 대주제 별로 분류
        for doc in docs:
            doc_lines = doc.split('\n') # 줄별로 나눠 구분
            doc_no = doc_lines[1][9:] # 1번째 줄의 기사 번호를 doc_no에 저장
            
            # #CAT'03 주제 정보 추출하여 doc_cat03에 입력
            doc_cat03 = ''
            for line in doc_lines[:10]:
                if line.startswith("#CAT'03:"):
                    doc_cat03 = line[10:]
                    break
            
            # 추출한 주제 별로 디렉토리 정리
            for cat_prefix in cat_prefixes:
                if doc_cat03.startswith(cat_prefix):
                    dir_index = cat_prefixes.index(cat_prefix)
                    break
                    
            # 문서 정보를 제거하고 기사 본문만 남기기
            filtered_lines = []
            for line in doc_lines:
                if not (line.startswith('#') or line.startswith('@')):
                    filtered_lines.append(line)
                    
            # 주제별 디렉토리에 기사를 파일로 쓰기
            filename = 'hkib-' + doc_no + '.txt'
            filepath = dir_prefix + target_dir + '/' + cat_dirs[dir_index]
            
            if not os.path.exists(filepath):
                os.makedirs(filepath)
            f = open(filepath + '/' + filename, 'w')
            f.write('\n'.join(filtered_lines))
            f.close()

In [0]:
# 대상이 되는 주제 폴더 선택
dirs = ['health', 'economy']

# 기사에 출현하는 단어와 레이블을 저장할 리스트를 생성
# 설명변수(단어)
x_ls = [] 
# 목적변수(레이블)
y_ls = []

tmp1 = [] # 명사만을 담을 배열 생성
tmp2 = ''

# 형태소 분석기 객체 생성
#tokenizer_han = Hannanum()
tokenizer = Kkma()

# 각 폴더의 파일을 하나씩 읽어들이며, 전처리 후 리스트에 저장
for i, d in enumerate(dirs):
    # 파일 목록 읽어오기
    files = os.listdir(dir_prefix + target_dir + '/' + d)
    
    for file in files:
        # 각 파일을 읽어들이기
        f = open(dir_prefix + target_dir + '/' + d + '/' + file, 'r', encoding='utf-8')
        raw = f.read()  
        
        # 정규표현식을 사용하여 불필요한 문자열을 제거한 다음 파일 내용을 출력
        reg_raw = re.sub(r'[-\'@#:/◆▲0-9a-zA-Z<>!-"*\(\)]', '', raw)
        reg_raw = re.sub(r'[ ]+', ' ', reg_raw)
        reg_raw = reg_raw.replace('\n', ' ')
        
        # 형태소분석을 거쳐 명사만을 추출한 리스트를 생성
        tokens = tokenizer.nouns(reg_raw)
        
        for token in tokens:
            tmp1.append(token)
            
        tmp2 = ' '.join(tmp1) # tmp1 리스트를 문자열로 변경 : ' ',join(list)
        x_ls.append(tmp2) # 각 기사별로 문자열 리스트를 x_ls 배열에 담는다.
        tmp1 = [] # tmp1 초기화
        
        # 기사 주제 레이블을 리스트에 저장
        y_ls.append(i)
        
        # 파일 닫기
        f.close()

In [7]:
# 데이터 프레임으로 변환해서 설명변수를 화면에 출력
pd.DataFrame(x_ls)

,0
0,보사부 불량 한약재 유통 내년 월 월이후 이후 한의원 등 한방 한방취급의료기관 취급...
1,대한 대한약품 대한약품공업협동조합 약품 공업 협동조합 일 우수 우수의약품 의약품 제...
2,한국 한국치과대학장 치과 대학장 협의회 협의회회장 회장 이의 이의웅 웅 연세대 학장...
3,년 전국민 전국민의료보험 의료 보험 실시 실시이후 이후 국내 의료기관중 기관 중 치...
4,민주당 일 오후 의원 의원회관 회관 소 소회의실 회의실 산업 산업안전보건 산업안전보...
...,...
6257,광고 광고대행전문회사 대행 전문 회사 동방 동방기획대표 기획 대표 남 남정휴은 정휴...
6258,종합 종합상사 상사 수출 엔고 선진국 선진국경기 경기 회복 회복등 등 올 증가세 일...
6259,분기 분기중 중 자동차 자동차수출 수출 지난해 기간 나 폭 발적 증가세 기록 전망 ...
6260,올 가전 가전제품 제품 내수 수출 일 업계 금년 분기 전자 전자사의 사의 억 억천만...


In [8]:
# 첫 번째 기사로부터 추출한 단어를 출력
print(x_ls[0])

보사부 불량 한약재 유통 내년 월 월이후 이후 한의원 등 한방 한방취급의료기관 취급 의료 기관 포장 성분 통일 규격 사용 보사 부 일 대 약전 생약 생약규격집 규격집 국내 백종 가운 데 사용빈도 빈도 녹용 우황 당귀 작약 개 품목 품 지정 고시 월 우수 우수품질관리기준 품질 관리 기준 시설 전 전문제 문제 약 약업소만이 업소 만이 생산 수 약국 한방취급 의료기관 규격품 비 비규격품 금지 이 전문제약업소 세척 세척건조절단포장 건조 절단 과정 품질검사 검사 실시 원산지 표시 밖 규격품목 범위 점진적 포장단위 단위 근 감초 일반 경우 백 고가 제한 방침 이번 조치 유통중인 중인 대다수 농가 재배 뒤 정제 정제후시중 후 시중 농산물 구분 가격 불안정 거래 서가 문란 문란하다 하다 지적 대상 감국 건강 계지 계피 곽 곽향 향 구기자 길 길경 경 녹각 녹 용 도인 마 마황 황 반하 복령 부자 사삼 산수유 산조인 산약 숙지황 시호 신곡 육계 령 진피 천궁 행인 향부자 황금 황기 황련 황백 후박


In [9]:
print(y_ls)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [10]:
# 설명변수와 목적변수를 NumPy 배열로 변환
x_array = np.array(x_ls)
y_array = np.array(y_ls)

# 단어 출현 횟수를 계수
cntvec = CountVectorizer()
x_cntvecs = cntvec.fit_transform(x_array)
x_cntarray = x_cntvecs.toarray()

# 데이터프레임으로 단어 출현 횟수 출력
pd.DataFrame(x_cntarray)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,102057,102058,102059,102060,102061,102062,102063,102064,102065,102066,102067,102068,102069,102070,102071,102072,102073,102074,102075,102076,102077,102078,102079,102080,102081,102082,102083,102084,102085,102086,102087,102088,102089,102090,102091,102092,102093,102094,102095,102096
0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6257,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6258,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6259,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6260,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [11]:
# 단어와 단어의 인덱스 표시
for k, v in sorted(cntvec.vocabulary_.items(), key=lambda x:x[1]):
    print(k, v)

ㄱ모씨 0
ㄱ씨 1
ㄴ모씨 2
ㄴ씨 3
ㅊ많 4
ㅏ이미지에 5
가가 6
가가치 7
가가치산업 8
가가치통신망 9
가감 10
가검물 11
가게 12
가격 13
가격간 14
가격거품 15
가격격차 16
가격결정 17
가격결정권 18
가격경 19
가격경쟁 20
가격경쟁력 21
가격경쟁력면 22
가격경쟁력약화 23
가격경쟁력향상 24
가격경쟁력향상외 25
가격경쟁력확보방안 26
가격경쟁력회복 27
가격경쟁전략 28
가격공세 29
가격관리 30
가격괴리 31
가격구조 32
가격구조때문 33
가격구조조정 34
가격권리행사가격 35
가격규제 36
가격급등 37
가격기구 38
가격기능 39
가격기준 40
가격담합 41
가격대 42
가격덤핑 43
가격동결 44
가격동결정책 45
가격동향울 46
가격동향조사 47
가격등 48
가격때문 49
가격만족도 50
가격메리트 51
가격면 52
가격명 53
가격문란 54
가격문제 55
가격배율 56
가격변동 57
가격변동성 58
가격변동제한폭 59
가격변동폭 60
가격변수 61
가격보조금 62
가격부담 63
가격부문 64
가격불균형 65
가격불문 66
가격불안 67
가격비교 68
가격비준표상 69
가격산정 70
가격산정프라이싱 71
가격산출 72
가격상 73
가격상승 74
가격상승률 75
가격상승세 76
가격상승억제요인 77
가격상승요인 78
가격상승지역 79
가격상승폭 80
가격선 81
가격소구 82
가격소매 83
가격수준 84
가격안전장치 85
가격안정 86
가격안정방향 87
가격안정사업자금 88
가격안정화 89
가격앙등 90
가격억제 91
가격왜곡해소 92
가격우선 93
가격우위 94
가격움직임 95
가격원상회 96
가격위험 97
가격으 98
가격이외 99
가격인상 100
가격인상러시현상 101
가격인상률 102
가격인상억제 103
가격인상업체 104
가격인상요인 105
가격인상폭 106
가격인상효과 107
가격인하 108
가격인하바람 109
가격인하전략 110
가격인하조치 111
가격입찰서 112
가격자유화 113
가격자유화조치 1

In [12]:
# 단어의 TF-IDF 계산
tfidf_vec = TfidfVectorizer(use_idf=True)
x_tfidf_vecs = tfidf_vec.fit_transform(x_array)
x_tfidf_array = x_tfidf_vecs.toarray()

# 데이터프레임으로 변환하여 단어의 출현 횟수를 출력
pd.DataFrame(x_tfidf_array)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,102057,102058,102059,102060,102061,102062,102063,102064,102065,102066,102067,102068,102069,102070,102071,102072,102073,102074,102075,102076,102077,102078,102079,102080,102081,102082,102083,102084,102085,102086,102087,102088,102089,102090,102091,102092,102093,102094,102095,102096
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.038184,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6257,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6258,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6259,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.048642,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.143657,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6260,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
# 데이터를 훈련 데이터와 테스트 데이터로 분할
train_X, test_X, train_Y, test_Y = model_selection.train_test_split(x_tfidf_array, y_array, test_size=0.2)

# 데이터 건수 확인
print(len(train_X))
print(len(test_X))

5009
1253


In [0]:
# PyTorch 라이브러리 임포트
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

In [15]:
# 훈련 데이터 텐서 생성
train_X = torch.from_numpy(train_X).float()
train_Y = torch.from_numpy(train_Y).long()

# 테스트 데이터 텐서 생성
test_X = torch.from_numpy(test_X).float()
test_Y = torch.from_numpy(test_Y).long()

# 텐서로 변환한 데이터 건수 확인
print(train_X.shape)
print(train_Y.shape)

torch.Size([5009, 102097])
torch.Size([5009])


In [16]:
# 설명변수와 목적변수의 텐서를 합친다
train = TensorDataset(train_X, train_Y)

# 첫번째 텐서 내용 확인
print(train[0])

# 미니배치 분할하기
train_loader = DataLoader(train, batch_size=100, shuffle=True)

(tensor([0., 0., 0.,  ..., 0., 0., 0.]), tensor(1))


In [0]:
# 신경망 구성
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(102097, 256)
        self.fc2 = nn.Linear(256, 256)
        self.fc3 = nn.Linear(256, 256)
        self.fc4 = nn.Linear(256, 128)
        self.fc5 = nn.Linear(128, 128)
        self.fc6 = nn.Linear(128, 2)
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = F.relu(self.fc5(x))
        x = self.fc5(x)
        return F.log_softmax(x)
    
# 인스턴스 생성
model = Net()

In [24]:
# 오차함수
criterion = nn.CrossEntropyLoss()

# 최적화 기법 선택
optimizer = optim.Adam(model.parameters(), lr=0.005)

# 학습
for epoch in range(20):
    total_loss = 0
    
    # 분할한 데이터(미니배치)를 차례로 꺼내옴
    for train_x, train_y in train_loader:
        # 계산 그래프 구성
        train_x, train_y = Variable(train_x), Variable(train_y)
        # 경사 초기화
        optimizer.zero_grad()
        # 순전파 계산
        output = model(train_x)
        # 오차 계산
        loss = criterion(output, train_y)
        # 역전파 계산
        loss.backward()
        # 가중치 업데이트
        optimizer.step()
        # 오차 누적 계산
        total_loss += loss.data
    print("epoch: %d, loss: %.6f" % (epoch+1,loss.data))

print('Finished Training')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


epoch: 1, loss: 0.607380
epoch: 2, loss: 0.000007
epoch: 3, loss: 0.000000
epoch: 4, loss: 0.000056
epoch: 5, loss: 0.000000
epoch: 6, loss: 0.000000
epoch: 7, loss: 0.000000
epoch: 8, loss: 0.000000
epoch: 9, loss: 0.000000
epoch: 10, loss: 0.000000
epoch: 11, loss: 0.000000
epoch: 12, loss: 0.000000
epoch: 13, loss: 0.000000
epoch: 14, loss: 0.000000
epoch: 15, loss: 0.000000
epoch: 16, loss: 0.000000
epoch: 17, loss: 0.000000
epoch: 18, loss: 0.000000
epoch: 19, loss: 0.000000
epoch: 20, loss: 0.000000
Finished Training


In [25]:
# 계산 그래프 구성
test_x, test_y = Variable(test_X), Variable(test_Y)

# 출력이 0 혹은 1이 되도록
result = torch.max(model(test_x).data, 1)[1]

# 모형의 정확도 계산
accuracy = sum(test_y.data.numpy() == result.numpy()) / len(test_y.data.numpy())

# 정확도 출력
accuracy

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


0.9792498004788508